# Program 4.12: Unrealized ideas for measuring polarization

This program was developed for the master's thesis “Agent-based modelling of embryonic organoid development” (Víctor Villegas-Morral, 2024) using [CellBasedModels.jl](https://github.com/dsb-lab/CellBasedModels.jl).

The report and the rest of the code can be found on the [Multiscale Physics of Living Systems Group’s GitHub](https://github.com/MPoLS-lab) and on [my personal GitHub](https://github.com/villegas-morral/masters-thesis).

### Initialization

In [ ]:
parameters = define_par();

dt = 0.002;
save_each = round(Int64, 0.25 / dt);
n_cells = 300;

### Aggregate

In [ ]:
Random.seed!(2345)
com = initialize_growth(parameters; dt);

In [ ]:
grow_size!(com, save_each, n_cells)
# grow_time!(com,save_each,30)
m0 = length(com);

In [ ]:
println(com.N)
println(formed_correctly(com))
plot_aggregate(com, color_map, 1, m0)

In [ ]:
# mechanics_evolve!(com, save_each, 30)
# m0 = length(com);

In [ ]:
growncom = deepcopy(com);

Let $ S_B $ represent the set of indices of cells in state **B**.

### 1.1 Define a distance-based metric

To determine whether the cells are evenly distributed or clustered, you can compute a function based on the pairwise distances between the cells in state B.

1. **Mean pairwise distance**:
   $$\text{Mean Distance} = \frac{2}{|S_B|(|S_B| - 1)} \sum_{\substack{i, j \in S_B \\ i \neq j}} \| x_i - x_j \|$$
   - This computes the average distance between all pairs of cells in state B. Higher values suggest a more even distribution, while lower values indicate clustering.

In [ ]:
function mean_dist(com, tstamp)
	x = com[tstamp].x
	y = com[tstamp].y
	z = com[tstamp].z
	N = com[tstamp].N

	mdist = 0
	for i in 1:N
		for j in i+1:N
			mdist += CBMMetrics.euclidean(x[i], x[j], y[i], y[j], z[i], z[j])
		end
	end
	mdist /= (N * (N - 1) / 2)

end;


In [ ]:
function mean_dist_state(com, tstamp, state)

	x = com[tstamp].x
	y = com[tstamp].y
	z = com[tstamp].z
	N = com[tstamp].N

	bcells = [i for i in 1:com[tstamp].N if com[tstamp].cell_state[i] == state]
	n_bcells = length(bcells)
	mdist = 0
	for i in bcells
		for j in bcells
			if i < j  # Avoid duplicate pairs and self-comparison
				mdist += CBMMetrics.euclidean(x[i], y[i], z[i], x[j], y[j], z[j])
			end
		end
	end
	mdist /= (n_bcells * (n_bcells - 1) / 2)

end;


2. **Variance of pairwise distances**:
   $$\text{Variance of Distances} = \frac{2}{|S_B|(|S_B| - 1)} \sum_{\substack{i, j \in S_B \\ i \neq j}} \left( \| x_i - x_j \| - \mu \right)^2$$
   where $ \mu $ is the mean pairwise distance. A **low variance** suggests that the cells are evenly distributed, while a **high variance** indicates that the cells are either clustered or unevenly spread.

In [ ]:
function mean_dist_state_var(com, tstamp, state)
	# tstamp = 150
	# state = 2

	x = com[tstamp].x
	y = com[tstamp].y
	z = com[tstamp].z
	N = com[tstamp].N
	mean = mean_dist(com, tstamp, state)

	bcells = [i for i in 1:com[tstamp].N if com[tstamp].cell_state[i] == state]
	n_bcells = length(bcells)
	var = 0
	for i in bcells
		for j in bcells
			if i < j  # Avoid duplicate pairs and self-comparison
				var += (CBMMetrics.euclidean(x[i], y[i], z[i], x[j], y[j], z[j])
						-
						mean)^2
			end
		end
	end
	var /= (n_bcells * (n_bcells - 1) / 2)
end
mean_dist_state_var(com, 129, 2)


### 1.2 Normalization and Final Function

To scale the function for different aggregates, you might want to normalize the distances by the size of the aggregate. For example, you can normalize by the average distance between the cells and the center of mass $ \text{cm} $.

$$d_{\text{avg}} = \frac{1}{|S_B|} \sum_{i \in S_B} \| x_i - \text{cm} \|$$

Then, a normalized measure of clustering can be:

$$F_B = \frac{\text{Mean Distance}}{d_{\text{avg}}}$$

- **If $ F_B $ is close to 1**, it indicates that the cells in state B are evenly distributed.
- **If $ F_B $ is much less than 1**, it suggests clustering.

In [ ]:
function cm_dist(com, tstamp)
	N = com[tstamp].N
	x = com[tstamp].x
	y = com[tstamp].y
	z = com[tstamp].z

	cm = [sum(x), sum(y), sum(z)] ./ N

	d_avg = 0
	for i in 1:N
		d_avg += CBMMetrics.euclidean(x[i], cm[1], y[i], cm[2], z[i], cm[3])
	end
	d_avg /= N
end;


In [ ]:
function cm_dist_state(com, tstamp)
end;

### 2. Alternative measure

Another option is to compute the following norm

$$ ||p(t)|| = ||\frac{1}{N_B}\sum_{i \in S_B}(x_i - x_c)|| $$

In [ ]:
function pol_measure(com, tstamp)
	N = com[tstamp].N
	x = com[tstamp].x
	y = com[tstamp].y
	z = com[tstamp].z
	xyz = collect(zip(x, y, z))

	cm = [sum(x), sum(y), sum(z)] ./ N

	pol_vec = Array{Float64}(undef, 3)
	for i in 1:N
		pol_vec .+= xyz[i] .- cm
	end
	cmdist = sqrt(pol_vec[1]^2 + pol_vec[2]^2 + pol_vec[3]^2) / N
	return cmdist
end;


In [ ]:
tstamp = 129
state = 2
function pol_measure_state(com, tstamp, state)
	N = com[tstamp].N
	x = com[tstamp].x
	y = com[tstamp].y
	z = com[tstamp].z
	xyz = collect(zip(x, y, z))

	cm = [sum(x), sum(y), sum(z)] ./ N
	bcells = [i for i in 1:com[tstamp].N if com[tstamp].cell_state[i] == state]
	nbcells = length(bcells)

	pol_vec = Array{Float64}(undef, 3)
	for i in bcells
		pol_vec .+= xyz[i] .- cm
	end
	cmdist = sqrt(pol_vec[1]^2 + pol_vec[2]^2 + pol_vec[3]^2) / nbcells
	return cmdist
end
pol_measure_state(com, 150, 2)


### Alternative: Nearest Neighbor Distance


Another simple approach is to use **nearest-neighbor distance**:

$$\text{Nearest Neighbor Distance} = \frac{1}{|S_B|} \sum_{i \in S_B} \min_{\substack{j \in S_B \\ j \neq i}} \| x_i - x_j \|$$

This value tends to be smaller when cells are clustered and larger when they are evenly distributed.